In [ ]:
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', None)  # Show all columns

In [ ]:
#df = pd.read_csv('/Users/z3/Documents/Studium/1. Semester/ProbML/Probabilistic-Machine-Learning_lecture-PROJECTS/projects/15-2BLXXXX_epa_census/EPA_SmartLocationDatabase_V3_Jan_2021_Final.csv')

In [ ]:
#selected_df = df[['GEOID20','STATEFP','COUNTYFP','CSA','CSA_Name','CBSA','CBSA_Name','CBSA_POP','CBSA_EMP','CBSA_WRK','TotPop','HH','P_WrkAge','Pct_AO0','Workers','R_PCTLOWWAGE','TotEmp','E_PctLowWage','D1A','D1B','D1C','D2A_JPHH','D3AAO','D4A','D4C']].rename(columns={
#'GEOID20': 'ID','STATEFP': 'STATE_ID','COUNTYFP': 'COUNTY_ID','CSA': 'CSA','CSA_Name': 'CSA_NAME','CBSA': 'CBSA','CBSA_Name': 'CBSA_NAME','CBSA_POP': 'CBSA_POP','CBSA_EMP': 'CBSA_EMP','CBSA_WRK': 'CBSA_WRK','TotPop': 'TOTAL_POPULATION','HH': 'HOUSEHOLDS','P_WrkAge': 'P_WORKING_AGE','Pct_AO0': 'P_ZERO_CARS_HOUSEHOLDS','Workers': 'WORKERS','R_PCTLOWWAGE': 'P_LOW_WAGE_WORKERS','TotEmp': 'TOTAL_EMPLOYMENT','E_PctLowWage': 'P_LOW_WAGE_EMPLOYMENT','D1A': 'HOUSEHOLD_P_ACRE','D1B': 'POPULATION_P_ACRE','D1C': 'JOBS_P_ACRE', 'D2A_JPHH' : 'JOBS_P_HOUSEHOLD','D3AAO': 'ROAD_NETWORK_MILES_AUTO','D4A': 'METERS_NEXT_TRANSIT_ STOP','D4C': 'TIMES_P_HOUR_TRANSIT_SERVICE'})
#selected_df['ID'] = selected_df['ID'].apply(lambda x: '{:.0f}'.format(x))

In [ ]:
#selected_df.to_csv('census_data', index=False) 

In [ ]:
# Randomly sample 10,000 rows
#sampled_df = selected_df.sample(n=10000, random_state=42)  # Set random_state for reproducibility

# Export to CSV
#sampled_df.to_csv('sampled_output.csv', index=False)
